# 选课信息

In [ ]:
username = ""
password = ""
driver_path = '' # webdriver路径
img_path = 'img' #图片暂存路径
gif_path = 'gif' #gif暂存路径

# 初始化

In [ ]:
from selenium import webdriver  # 用来驱动浏览器的
from selenium.webdriver.support import expected_conditions as EC  # 和下面WebDriverWait一起用的
from selenium.webdriver.support.wait import WebDriverWait  # 等待页面加载某些元素
import pyautogui
import time
import requests
import os
from PIL import Image
import base64
import json
import requests
from selenium.webdriver.chrome.options import Options
from random import random
#chrome_options = Options()
#chrome_options.add_argument('--headless')
#driver = webdriver.Chrome(driver_path,chrome_options = chrome_options)
#driver = webdriver.Chrome(driver_path)

In [ ]:
def ran(x):
    return x+(random()-0.5)*x

# 自动选课脚本

In [ ]:
class elective:
    def __init__(self,driver_path):
        self.driver = webdriver.Chrome(driver_path)
    
    # 登录
    def login(self,username,password):
        self.driver.get('https://elective.pku.edu.cn/')
        self.driver.implicitly_wait(10)
        self.driver.find_element_by_id('user_name').send_keys(username)
        self.driver.find_element_by_id('password').send_keys(password)
        self.driver.find_element_by_id('logon_button').click()
        self.driver.implicitly_wait(10)
        self.driver.find_element_by_link_text('补退选').click()
        self.driver.implicitly_wait(10)

        
    # 创建临时目录
    def clear_dir(self,path):
        if not os.path.isdir(path):
            os.mkdir(path)  # 创建目录
        else:  # 清空目录
            [os.remove(os.path.join(path, file_name)) for file_name in os.listdir(path)]
    
    
    # 获取验证码
    def get_captcha(self,img_path):
        
        # 清空路径
        self.clear_dir(img_path)
        self.driver.implicitly_wait(10)
        # 预截屏5次，避免刚开始截屏时卡顿
        tem_path = os.path.join(img_path,'1.png')
        for i in range(5):
            self.driver.find_element_by_id('imgname').screenshot(tem_path)
        
        # 点击“换一个”
        self.driver.find_element_by_link_text('换一个').click()
        self.driver.implicitly_wait(10)
        
        # 截屏
        for i in range(200):
            img_file = os.path.join(img_path,str(i)+'.png')
            self.driver.find_element_by_id('imgname').screenshot(img_file)
            
            
    # 输入验证码
    def send_captcha(self,captcha):
        self.driver.find_element_by_id('validCode').send_keys(captcha)
        
    
    # 刷新或补选
    def refresh_or_elect(self):
        try:
            self.driver.find_element_by_link_text('刷新').click()
            time.sleep(ran(0.2))
            self.driver.switch_to.alert.accept()
        except :
            try:
                self.driver.find_element_by_link_text('补选').click()
                time.sleep(ran(0.2))
                self.driver.switch_to.alert.accept()
            except:
                pass
    
    # 直接补选
    def elect(self):
        self.driver.find_element_by_link_text('补选').click()
        time.sleep(ran(0.2))
        self.driver.switch_to.alert.accept()
        
    def close(self):
        self.driver.close()

# 获取验证码

## 合成gif

In [ ]:
def make_gif(img_path):
    img_list = os.listdir(img_path)
    img_list.sort(key = lambda x:int(x.split('.')[0]))
    first_img = Image.open(os.path.join(img_path, img_list[0]))  # 第一张图片对象
    else_imgs = [Image.open(os.path.join(img_path, img)) for img in img_list[1:]]  # 剩余图片对象

    first_img.save("record.gif", append_images=else_imgs,
                 duration=150,  # 每张图片的过过渡时间
                 save_all=True) # 拼接保存，如果想要循环播放可以加上loop=0

## 利用ttshitu.com识别验证码

In [ ]:
def base64_api(uname, pwd, img, typeid):
    with open(img, 'rb') as f:
        base64_data = base64.b64encode(f.read())
        b64 = base64_data.decode()
    data = {"username": uname, "password": pwd, "typeid": typeid, "image": b64}
    result = json.loads(requests.post("http://api.ttshitu.com/predict", json=data).text)
    if result['success']:
        return result["data"]["result"]
    else:
        return result["message"]
    return ""

## 识别验证码

In [ ]:
def recognize(img_path):
    make_gif(img_path)
    captcha = base64_api('','','record.gif',7)
    return captcha

# 主程序

In [ ]:
while True:
    try:
        # 生成
        elective_lst = []
        for i in range(4):
            elective_lst.append(elective(driver_path))
            
        # 登录    
        for x in elective_lst:
            x.login(username,password)
            time.sleep(ran(1))
        
        # 填充验证码
        for x in elective_lst:
            x.get_captcha(img_path)
            captcha = recognize(img_path)
            x.send_captcha(captcha)
        
        
        # 刷新
        for j in range(5):
            for x in elective_lst:
                x.refresh_or_elect()
                #x.elect()
                time.sleep(ran(2))
        
        # 关闭
        for x in elective_lst:
            x.close()
        
    except:
        pass